Start by importing `Magma` and `Mantle`.

In [1]:
import magma as m

import os
os.environ['MANTLE_TARGET'] = 'ice40'
import mantle

import mantle lattice ice40
import mantle lattice mantle40


Now let's build a 4-bit adder from the FullAdder.

In [2]:
class FullAdder(m.Circuit):
    name = "FullAdder"
    IO = ["A", m.In(m.Bit), "B", m.In(m.Bit), "Cin", m.In(m.Bit), "Sum", m.Out(m.Bit), "Cout", m.Out(m.Bit)]
    @classmethod
    def definition(io):
        # Compute the sum
        sum_ = io.A ^ io.B ^ io.Cin
        m.wire(sum_, io.Sum)
        # Compute the carry
        cout = (io.A & io.B) | (io.B & io.Cin) | (io.A & io.Cin)
        # Wire cout
        m.wire(cout, io.Cout)

In [3]:
def adder(A, B):
    n = len(A)
    C = []
    Cout = 0
    for i in range(n):
        fulladder = FullAdder()
        m.wire(A[i], fulladder.A)
        m.wire(B[i], fulladder.B)
        m.wire(Cout, fulladder.Cin)
        C.append(fulladder.Sum)
        Cout = fulladder.Cout
    return m.uint(C)  

Initialize the IceStick headers. `J1` will be used as inputs; `J3` will be used as outputs.

In [4]:
from loam.boards.icestick import IceStick

icestick = IceStick()
for i in range(4):
    icestick.J1[i].input().on()
    icestick.J1[i+4].input().on()
    icestick.J3[i].output().on()

In [5]:
main = icestick.DefineMain()

m.wire(adder(main.J1[0:4], main.J1[4:8]), main.J3)

m.EndDefine()

In [6]:
m.compile('build/adder', main)

compiling XOr2
compiling And2
compiling Or2
compiling FullAdder
compiling main


In [7]:
%%bash
yosys -q -p 'synth_ice40 -top main -blif build/adder.blif' build/adder.v
arachne-pnr -q -d 1k -o build/adder.txt -p build/adder.pcf build/adder.blif 
icepack build/adder.txt build/adder.bin

In [8]:
%%bash 
# iceprog build/adder.bin